# MED-PC Extracting the Recording Data and Metadata

## Importing the Python Libraries

In [1]:
import sys
import glob
from collections import defaultdict
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from moviepy.editor import *
from datetime import datetime
from IPython.display import Video
from pathlib import Path

In [2]:
from medpc2excel.medpc_read import medpc_read
# import cv2


In [3]:
# setting path
sys.path.append('../../src')

In [4]:
# All the libraries that were created for this repository
import extract.dataframe
import processing.tone
import extract.metadata

In [5]:
from functools import reduce

# Make a dataframe of all the starting frames

In [6]:
all_comp_to_alone = sorted(glob.glob("/scratch/back_up/reward_competition_extention/proc/reencoded_videos/*training*/*1.fixed.mp4"))

In [7]:
all_comp_to_alone

['/scratch/back_up/reward_competition_extention/proc/reencoded_videos/20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3/20230612_101430_standard_comp_to_training_D1_subj_1-4_and_1-3.1.fixed.mp4',
 '/scratch/back_up/reward_competition_extention/proc/reencoded_videos/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.fixed.mp4',
 '/scratch/back_up/reward_competition_extention/proc/reencoded_videos/20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1/20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1.fixed.mp4',
 '/scratch/back_up/reward_competition_extention/proc/reencoded_videos/20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4/20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1.fixed.mp4',
 '/scratch/back_up/reward_competition_extention/proc/reencoded_videos/20230613_110126_standard_comp_to_training_D2_subj_2-4_and_2-2/20230613_110126_standard_comp_to_tra

In [8]:
all_video_df = pd.read_excel("./data/rce_video_log.xlsx", header=0)

In [9]:
all_video_df

,video_file,first_tone_frame,second_tone_frame,contains_all_comp,first_and_second_tone_difference
0,/scratch/back_up/reward_competition_extention/...,1126,3529,True,2403
1,/scratch/back_up/reward_competition_extention/...,1125,3521,True,2396
2,/scratch/back_up/reward_competition_extention/...,918,3313,True,2395
3,/scratch/back_up/reward_competition_extention/...,1106,3513,True,2407
4,/scratch/back_up/reward_competition_extention/...,856,3252,True,2396
5,/scratch/back_up/reward_competition_extention/...,1021,3428,True,2407
6,/scratch/back_up/reward_competition_extention/...,89,2484,True,2395
7,/scratch/back_up/reward_competition_extention/...,536,2932,True,2396


In [10]:
all_video_df["fps"] = all_video_df["video_file"].apply(lambda x: VideoFileClip(x).fps)

In [11]:
all_video_df

,video_file,first_tone_frame,second_tone_frame,contains_all_comp,first_and_second_tone_difference,fps
0,/scratch/back_up/reward_competition_extention/...,1126,3529,True,2403,30.0
1,/scratch/back_up/reward_competition_extention/...,1125,3521,True,2396,30.0
2,/scratch/back_up/reward_competition_extention/...,918,3313,True,2395,29.0
3,/scratch/back_up/reward_competition_extention/...,1106,3513,True,2407,26.0
4,/scratch/back_up/reward_competition_extention/...,856,3252,True,2396,23.0
5,/scratch/back_up/reward_competition_extention/...,1021,3428,True,2407,23.0
6,/scratch/back_up/reward_competition_extention/...,89,2484,True,2395,29.0
7,/scratch/back_up/reward_competition_extention/...,536,2932,True,2396,28.0


In [12]:
all_itis_in_seconds = [120, 115, 90, 55, 50, 95, 75, 105, 50, 80, 65, 65, 50, 70, 95, 80, 70, 110, 60, 300, 60, 55, 65, 55, 50, 50, 105, 120, 120, 110, 85, 120, 60, 65, 60, 75, 100, 95, 50]

In [13]:
all_video_df["all_itis_in_frames"] = all_video_df["fps"].apply(lambda x: [int(20 * iti) for iti in all_itis_in_seconds])

In [14]:
all_video_df

,video_file,first_tone_frame,second_tone_frame,contains_all_comp,first_and_second_tone_difference,fps,all_itis_in_frames
0,/scratch/back_up/reward_competition_extention/...,1126,3529,True,2403,30.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210..."
1,/scratch/back_up/reward_competition_extention/...,1125,3521,True,2396,30.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210..."
2,/scratch/back_up/reward_competition_extention/...,918,3313,True,2395,29.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210..."
3,/scratch/back_up/reward_competition_extention/...,1106,3513,True,2407,26.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210..."
4,/scratch/back_up/reward_competition_extention/...,856,3252,True,2396,23.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210..."
5,/scratch/back_up/reward_competition_extention/...,1021,3428,True,2407,23.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210..."
6,/scratch/back_up/reward_competition_extention/...,89,2484,True,2395,29.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210..."
7,/scratch/back_up/reward_competition_extention/...,536,2932,True,2396,28.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210..."


In [16]:
for file in all_video_df["video_file"]:
    print(file)

/scratch/back_up/reward_competition_extention/proc/reencoded_videos/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.fixed.mp4
/scratch/back_up/reward_competition_extention/proc/reencoded_videos/20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1/20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1.fixed.mp4
/scratch/back_up/reward_competition_extention/proc/reencoded_videos/20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4/20230613_105657_standard_comp_to_training_D2_subj_1-1_and_1-4.1.fixed.mp4
/scratch/back_up/reward_competition_extention/proc/reencoded_videos/20230613_110126_standard_comp_to_training_D2_subj_2-4_and_2-2/20230613_110126_standard_comp_to_training_D2_subj_2-4_and_2-2.1.fixed.mp4
/scratch/back_up/reward_competition_extention/proc/reencoded_videos/20230614_114041_standard_comp_to_training_D3_subj_1-1_and_1-2/20230614_114041_standard_comp_to_training_D3_subj_1-1_

In [17]:
# Your initial list
lst = [1, 2, 3, 4, 5]

# Use reduce to generate a list of running totals
result = reduce(lambda acc, x: acc + [acc[-1] + x] if acc else [x], lst, [])


In [18]:
def get_running_total(lst):
    return reduce(lambda acc, x: acc + [acc[-1] + x] if acc else [x], lst, [])

all_video_df["all_tone_frame"] = all_video_df["first_tone_frame"].apply(lambda x: get_running_total([x] + all_itis_in_frames))

In [19]:
all_video_df["all_tone_frame"] = all_video_df.apply(lambda row: get_running_total([row["first_tone_frame"]] + row["all_itis_in_frames"]), axis=1)

In [20]:
all_video_df

,video_file,first_tone_frame,second_tone_frame,contains_all_comp,first_and_second_tone_difference,fps,all_itis_in_frames,all_tone_frame
0,/scratch/back_up/reward_competition_extention/...,1126,3529,True,2403,30.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[1126, 3526, 5826, 7626, 8726, 9726, 11626, 13..."
1,/scratch/back_up/reward_competition_extention/...,1125,3521,True,2396,30.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[1125, 3525, 5825, 7625, 8725, 9725, 11625, 13..."
2,/scratch/back_up/reward_competition_extention/...,918,3313,True,2395,29.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[918, 3318, 5618, 7418, 8518, 9518, 11418, 129..."
3,/scratch/back_up/reward_competition_extention/...,1106,3513,True,2407,26.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[1106, 3506, 5806, 7606, 8706, 9706, 11606, 13..."
4,/scratch/back_up/reward_competition_extention/...,856,3252,True,2396,23.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[856, 3256, 5556, 7356, 8456, 9456, 11356, 128..."
5,/scratch/back_up/reward_competition_extention/...,1021,3428,True,2407,23.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[1021, 3421, 5721, 7521, 8621, 9621, 11521, 13..."
6,/scratch/back_up/reward_competition_extention/...,89,2484,True,2395,29.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[89, 2489, 4789, 6589, 7689, 8689, 10589, 1208..."
7,/scratch/back_up/reward_competition_extention/...,536,2932,True,2396,28.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[536, 2936, 5236, 7036, 8136, 9136, 11036, 125..."


In [21]:
all_video_df["all_5sec_before_tone"] = all_video_df.apply(lambda row: [int(i - 10 * 20) for i in row["all_tone_frame"]], axis=1)
# all_video_df["all_5sec_before_tone"] = all_video_df.apply(lambda row: [int(i - 10 * row["fps"]) for i in row["all_tone_frame"]], axis=1)

In [22]:
all_video_df["all_10sec_after_tone"] = all_video_df.apply(lambda row: [int(i + 60 * 20) for i in row["all_tone_frame"]], axis=1)
# all_video_df["all_5sec_before_tone"] = all_video_df.apply(lambda row: [int(i - 10 * row["fps"]) for i in row["all_tone_frame"]], axis=1)

In [23]:
all_video_df

,video_file,first_tone_frame,second_tone_frame,contains_all_comp,first_and_second_tone_difference,fps,all_itis_in_frames,all_tone_frame,all_5sec_before_tone,all_10sec_after_tone
0,/scratch/back_up/reward_competition_extention/...,1126,3529,True,2403,30.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[1126, 3526, 5826, 7626, 8726, 9726, 11626, 13...","[926, 3326, 5626, 7426, 8526, 9526, 11426, 129...","[2326, 4726, 7026, 8826, 9926, 10926, 12826, 1..."
1,/scratch/back_up/reward_competition_extention/...,1125,3521,True,2396,30.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[1125, 3525, 5825, 7625, 8725, 9725, 11625, 13...","[925, 3325, 5625, 7425, 8525, 9525, 11425, 129...","[2325, 4725, 7025, 8825, 9925, 10925, 12825, 1..."
2,/scratch/back_up/reward_competition_extention/...,918,3313,True,2395,29.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[918, 3318, 5618, 7418, 8518, 9518, 11418, 129...","[718, 3118, 5418, 7218, 8318, 9318, 11218, 127...","[2118, 4518, 6818, 8618, 9718, 10718, 12618, 1..."
3,/scratch/back_up/reward_competition_extention/...,1106,3513,True,2407,26.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[1106, 3506, 5806, 7606, 8706, 9706, 11606, 13...","[906, 3306, 5606, 7406, 8506, 9506, 11406, 129...","[2306, 4706, 7006, 8806, 9906, 10906, 12806, 1..."
4,/scratch/back_up/reward_competition_extention/...,856,3252,True,2396,23.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[856, 3256, 5556, 7356, 8456, 9456, 11356, 128...","[656, 3056, 5356, 7156, 8256, 9256, 11156, 126...","[2056, 4456, 6756, 8556, 9656, 10656, 12556, 1..."
5,/scratch/back_up/reward_competition_extention/...,1021,3428,True,2407,23.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[1021, 3421, 5721, 7521, 8621, 9621, 11521, 13...","[821, 3221, 5521, 7321, 8421, 9421, 11321, 128...","[2221, 4621, 6921, 8721, 9821, 10821, 12721, 1..."
6,/scratch/back_up/reward_competition_extention/...,89,2484,True,2395,29.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[89, 2489, 4789, 6589, 7689, 8689, 10589, 1208...","[-111, 2289, 4589, 6389, 7489, 8489, 10389, 11...","[1289, 3689, 5989, 7789, 8889, 9889, 11789, 13..."
7,/scratch/back_up/reward_competition_extention/...,536,2932,True,2396,28.0,"[2400, 2300, 1800, 1100, 1000, 1900, 1500, 210...","[536, 2936, 5236, 7036, 8136, 9136, 11036, 125...","[336, 2736, 5036, 6836, 7936, 8936, 10836, 123...","[1736, 4136, 6436, 8236, 9336, 10336, 12236, 1..."


## Creating the clips

In [24]:
for index, row in all_video_df.iterrows():
    video_basename = os.path.basename(row["video_file"].strip("fixed.mp4"))    
    video_output_dir = os.path.join("/scratch/back_up/reward_competition_extention/proc/tone_clips", video_basename)
    os.makedirs(video_output_dir, exist_ok=True)
                
    for index, (first_frame, last_frame) in enumerate(zip(row["all_5sec_before_tone"], row["all_10sec_after_tone"])):
        if first_frame <= 0:
            first_frame = 0

        try:

            # print(first_frame, last_frame)
            clip_output_path = os.path.join(video_output_dir, "{}.tone_{}.mp4".format(video_basename, index + 1))
            if not os.path.isfile(clip_output_path):
                video_object = VideoFileClip(row["video_file"])
                print(first_frame, last_frame)
                cs_trimmed_video = video_object.subclip(first_frame//30, last_frame//30)
                cs_trimmed_video.to_videofile(clip_output_path)
            else:
                print("Tone Clip exists for: {}".format(clip_output_path))
                continue

        except Exception as e: 
            print(e)           

926 2326
Moviepy - Building video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.tone_1.mp4.
Moviepy - Writing video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.tone_1.mp4



Moviepy - Done !
Moviepy - video ready /scratch/back_up/reward_competition_extention/proc/tone_clips/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.tone_1.mp4
3326 4726
Moviepy - Building video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.tone_2.mp4.
Moviepy - Writing video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.tone_2.mp4



Moviepy - Done !
Moviepy - video ready /scratch/back_up/reward_competition_extention/proc/tone_clips/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.tone_2.mp4
5626 7026
Moviepy - Building video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.tone_3.mp4.
Moviepy - Writing video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.tone_3.mp4



Moviepy - Done !
Moviepy - video ready /scratch/back_up/reward_competition_extention/proc/tone_clips/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.tone_3.mp4
7426 8826
Moviepy - Building video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.tone_4.mp4.
Moviepy - Writing video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.tone_4.mp4



Moviepy - Done !
Moviepy - video ready /scratch/back_up/reward_competition_extention/proc/tone_clips/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.tone_4.mp4
8526 9926
Moviepy - Building video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.tone_5.mp4.
Moviepy - Writing video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1/20230612_101935_standard_comp_to_training_D1_subj_2-2_and_2-1.1.tone_5.mp4



...

Moviepy - Done !
Moviepy - video ready /scratch/back_up/reward_competition_extention/proc/tone_clips/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1.tone_36.mp4
61436 62836
Moviepy - Building video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1.tone_37.mp4.
Moviepy - Writing video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1.tone_37.mp4



Moviepy - Done !
Moviepy - video ready /scratch/back_up/reward_competition_extention/proc/tone_clips/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1.tone_37.mp4
63436 64836
Moviepy - Building video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1.tone_38.mp4.
Moviepy - Writing video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1.tone_38.mp4



Moviepy - Done !
Moviepy - video ready /scratch/back_up/reward_competition_extention/proc/tone_clips/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1.tone_38.mp4
65336 66736
Moviepy - Building video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1.tone_39.mp4.
Moviepy - Writing video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1.tone_39.mp4



Moviepy - Done !
Moviepy - video ready /scratch/back_up/reward_competition_extention/proc/tone_clips/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1.tone_39.mp4
66336 67736
Moviepy - Building video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1.tone_40.mp4.
Moviepy - Writing video /scratch/back_up/reward_competition_extention/proc/tone_clips/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1.tone_40.mp4



Moviepy - Done !
Moviepy - video ready /scratch/back_up/reward_competition_extention/proc/tone_clips/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1/20230616_111904_standard_comp_to_training_D4_subj_2-1_and_2-2.1.tone_40.mp4


In [25]:
raise ValueError()

ValueError: 

In [ ]:
for index, row in all_cs_presentation_dataframe.iterrows():
    os.makedirs(row["tone_clip_output_dirname"], exist_ok=True)
    video_object = VideoFileClip(row["video_full_path"])
    cs_trimmed_video = video_object.subclip(row["10_seconds_before_cs_presentation_in_seconds"], row["10_seconds_after_cs_presentation_in_seconds"])
    cs_trimmed_video.to_videofile(row["tone_clip_output_full_path"])

# OLD WORK BELOW

In [ ]:
all_video_df

In [ ]:
(3529-1126)/120

In [ ]:
(5830 - 3529)/115

In [ ]:
(7627 - 5830)/90

In [ ]:
(8726 - 7627)/55

In [ ]:
all_itis = [80, 90, 80, 75, 100, 95, 90, 80, 90, 100, 90, 120, 90, 85, 90, 95, 120, 80, 95, 80, 90, 80, 75, 100, 95, 90, 80, 90, 100, 90, 120, 90, 85, 90, 95, 90, 80, 90, 100, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90 ]

In [ ]:
all_itis = [120, 115, 90, 55, 50, 95, 75, 105, 50, 80, 65, 65, 50, 70, 95, 80, 70, 110, 60, 300, 60, 55, 65, 55, 50, 50, 105, 120, 120, 110, 85, 120, 60, 65, 60, 75, 100, 95, 50]

In [ ]:
current_frame = 1126
all_tone_frames = [current_frame]
for iti in all_itis:
    current_frame += iti * 20
    all_tone_frames.append(current_frame)

In [ ]:
current_frame = 918
all_tone_frames = [current_frame]
for iti in all_itis:
    current_frame += iti * 20
    all_tone_frames.append(current_frame)

In [ ]:
all_tone_frames

1125, 3521, 5816, 7613 for 20230612_112630_standard_comp_to_training_D1_subj_1-2_and_1-1.1.fixed.mp4

## Getting the Metadata from all the MED-PC Recording Files

- Getting all the file paths of the recording files(that happen to all end in `.txt`

# NOTE: The following path must be changed to the directory where your MED-PC recording files are located, if they are not in the specied folder

In [ ]:
all_med_pc_file = sorted(glob.glob("./data/**/*.txt", recursive=True))

- Use this instead if you're using you're own data

In [ ]:
all_med_pc_file[:10]

- Selecting only one of the files
    - The tone times are the same for both C57 and CD1s

In [ ]:
all_med_pc_file[0]

- Example of what the MED-PC Recording file looks like

In [ ]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

- We will be extracting the first 10 or so lines that look like:

```
File: C:\MED-PC\Data\2022-05-06_12h59m_Subject 3.4 (2).txt

Start Date: 05/06/22

End Date: 05/06/22

Subject: 3.4 (2)

Experiment: Pilot of Pilot

Group: Cage 4

Box: 1

Start Time: 12:59:58

End Time: 14:02:38

MSN: levelNP_CS_reward_laserepochON1st_noshock
```
    
- We will just find all the lines that start with "File", "Start Date", "End Date", "Subject", "Experiment", "Group", "Box", "Start Time", "End Time", or "MSN". And then stop once all the metadata types have been collected

In [ ]:
# This makes a nested dictionary of file paths to each individual metadata type
file_path_to_meta_data = extract.metadata.get_all_med_pc_meta_data_from_files(list_of_files=all_med_pc_file)

In [ ]:
for key, value in file_path_to_meta_data.items():
    print("File path: {}".format(key))
    print("Metadata types and associated values: {}".format(value))
    break

## Making a Dataframe out of the Metadata

In [ ]:
# Turning the dictionary into a Pandas Dataframe
metadata_df = pd.DataFrame.from_dict(file_path_to_meta_data, orient="index")
# Resetting the index because currently the file path is the index 
metadata_df = metadata_df.reset_index()

In [ ]:
metadata_df.head()

In [ ]:
metadata_df.tail()

- Just getting the numbers out of the column that contains the cage information

## Inputting all the MED-PC log files

- Example of what the MED-PC Script looks like that was ran when recording the behaviors

In [ ]:
med_pc_scripts = glob.glob("./data/**/*.MPC")

In [ ]:
med_pc_scripts

In [ ]:
with open(med_pc_scripts[0]) as f:
    lines = f.readlines()
    for line in lines[:100]:
        print(line)

- We will be using the comments in the MED-PC script(which starts with the `\`) to create a name for the variables. By default, MED-PC uses a single letter as the name of the variable.
    - This will use the medpc2excel library found in https://github.com/cyf203/medpc2excel
- Example of the comments in the MED-PC script that we will use the names from:

```
\Variables

\A - Time since last CS

\B - Shock intensity

\C - Counter array

\D - Current ITI value

\E - CS ITI values for first few trials

\F - Shock duration

\G -

\H -

\I -

\J - Shock intensity repo

\K - CS type

\L -

\M - CS type repo

\N - Port exit time stamp array

\O -

\P - Port entry time stamp array

\Q - Sucrose delivery time stamp array (absolute)

\R - Sucrose delivery time stamp array (relative to last CS)

\S - CS presentation time stamp array

\T - Session timer

\U - Time since last CS presentation

\V - List of CS ITI values (tone + houselight)

\W - ITI values used for CS one each trial

\X -

\Y - Beam break monitor variable

\Z -
```

In [ ]:
with open(all_med_pc_file[0]) as f:
    lines = f.readlines()
    for line in lines[:20]:
        print(line)

- **Please make sure that the corresponding `.mpc` file (aka the MED-PC script) that was ran to create the log file, is also in the same folder**

In [ ]:
all_med_pc_file

In [ ]:
concatted_medpc_df = extract.dataframe.get_medpc_dataframe_from_list_of_files(medpc_files=[all_med_pc_file[0]])
concatted_medpc_df = concatted_medpc_df.reset_index(drop=True)

In [ ]:
concatted_medpc_df.head()

In [ ]:
concatted_medpc_df.tail()

- Dropping all rows that don't have the CS presentation time

In [ ]:
cs_time_df = concatted_medpc_df[concatted_medpc_df["(S)CSpresentation"] != 0.0].dropna(subset="(S)CSpresentation")

In [ ]:
cs_time_df = cs_time_df[cs_time_df["(S)CSpresentation"] % 100 != 0]

In [ ]:
cs_time_df["file_path"].unique()

In [ ]:
cs_time_df

In [ ]:
cs_time_df["cs_minute_only"] = cs_time_df["(S)CSpresentation"] // 60
cs_time_df["cs_second_only"] = cs_time_df["(S)CSpresentation"] % 60


## Getting a list of the Tone presentation times

In [ ]:
cs_presentation_time = cs_time_df[["(S)CSpresentation", "cs_minute_only", "cs_second_only"]].copy()

In [ ]:
cs_presentation_time

# Importing Video Data

In [ ]:
from IPython.display import Video

1. Play the video, and go to time where the first tone is played in the dataframe above
    - Usually 1 minute
2. Note when the first tone is playing in the video
3. Subtract the difference between the time that the tone is playing in the video and in the dataframe
    - i.e. 1 minute 12 seconds - 1 minute = 12 seconds

In [ ]:
video_file_paths = sorted(glob.glob("../2023_01_24_rc_sleap_c57/data/fixed_reward_competition_D1_videos/10-03-22_Test_16_1-1v1-3.fixed.mp4", recursive=True))

In [ ]:
for index, file_path in enumerate(video_file_paths):
    print("Index: {} File Path: {}".format(index, file_path))

In [ ]:
video_path = video_file_paths[0]

In [ ]:
video_path

## Calculating the predicted tone times in the video based on the difference

# NOTE: Change the value of the difference below

In [ ]:
cs_time_difference = 11

In [ ]:
cs_presentation_time["video_cs_time"] = cs_presentation_time["(S)CSpresentation"] + cs_time_difference

In [ ]:
cs_presentation_time

In [ ]:
cs_presentation_time["video_cs_minute_only"] = cs_presentation_time["video_cs_time"] // 60
cs_presentation_time["video_cs_second_only"] = cs_presentation_time["video_cs_time"] % 60


# Converting the tone time to frames

- Getting the FPS of the video

In [ ]:
cam = cv2.VideoCapture(video_path)
fps = cam.get(cv2.CAP_PROP_FPS)

In [ ]:
fps

In [ ]:
2008/fps

- Calculating the frame number by multiplying the FPS by the time in the video

In [ ]:
cs_presentation_time["video_cs_frame"] = (cs_presentation_time["video_cs_time"] * fps).astype(int)

In [ ]:
cs_presentation_time.head()

In [ ]:
cs_presentation_time.tail()

In [ ]:
cs_presentation_time.to_csv("./proc/cs_presentation_time.csv")

In [ ]:
raise ValueError()

## Getting the estimated tone time

In [ ]:
video_file_name_to_sub_dicts = defaultdict(dict)

In [ ]:
video_file_name_to_sub_dicts["10-03-22_Test_13_2-2v2-4.mp4"]["delay_in_seconds"] = 11
video_file_name_to_sub_dicts["10-03-22_Test_25_1-2v1-3.mp4"]["delay_in_seconds"] = 8
video_file_name_to_sub_dicts["10-03-22_Test_29_3-2v3-3.mp4"]["delay_in_seconds"] = 7
video_file_name_to_sub_dicts["10-03-22_Test_22_6-1v6-3.mp4"]["delay_in_seconds"] = 9
video_file_name_to_sub_dicts["10-03-22_Test_32_4-1v4-4.mp4"]["delay_in_seconds"] = 7
video_file_name_to_sub_dicts["10-03-22_Test_34_5-1v5-4.mp4"]["delay_in_seconds"] = 7

In [ ]:
video_file_name_to_sub_dicts

In [ ]:
cs_presentation_time = cs_time_df[["(S)CSpresentation", "cs_minute_only", "cs_second_only"]].copy()

In [ ]:
for video_relative_path in video_file_paths:
    video_file_name = os.path.basename(video_relative_path)
    video_full_path = os.path.abspath(video_relative_path)
    if video_file_name in video_file_name_to_sub_dicts:
        copy_cs_presentation_time = cs_presentation_time.copy()
        copy_cs_presentation_time["video_file_name"] = video_file_name
        copy_cs_presentation_time["video_full_path"] = video_full_path
        copy_cs_presentation_time["estimated_cs_time_in_seconds"] = copy_cs_presentation_time["(S)CSpresentation"] + video_file_name_to_sub_dicts[video_file_name]["delay_in_seconds"]
        
        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"] = copy_cs_presentation_time

In [ ]:
video_file_name

In [ ]:
video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]

# Doing this for all videos

## Calculating the estimated tone frame time

In [ ]:
video_file_paths

In [ ]:
for video_relative_path in video_file_paths:
    video_file_name = os.path.basename(video_relative_path)
    video_full_path = os.path.abspath(video_relative_path)

    print(video_file_name)

    if video_file_name in video_file_name_to_sub_dicts:
        copy_cs_presentation_time = cs_presentation_time.copy()
        copy_cs_presentation_time["video_file_name"] = video_file_name
        copy_cs_presentation_time["video_full_path"] = video_full_path
        copy_cs_presentation_time["estimated_cs_time_in_seconds"] = copy_cs_presentation_time["(S)CSpresentation"] + video_file_name_to_sub_dicts[video_file_name]["delay_in_seconds"]
        copy_cs_presentation_time.insert(0, 'tone_number', range(1, 1 + len(copy_cs_presentation_time)))

        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"] = copy_cs_presentation_time
        
        
        
        cv2_video_object = cv2.VideoCapture(video_relative_path)
        fps = cv2_video_object.get(cv2.CAP_PROP_FPS)
        video_file_name_to_sub_dicts[video_file_name]["fps"] = fps
        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["fps"] = fps
        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["estimated_cs_time_in_frames"] = (video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["estimated_cs_time_in_seconds"] * fps)

In [ ]:
video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]

# Adding the actual frame times

In [ ]:
video_file_name_to_sub_dicts["10-03-22_Test_13_2-2v2-4.mp4"]["first_cs_presentation_frame"] = 1771
video_file_name_to_sub_dicts["10-03-22_Test_25_1-2v1-3.mp4"]["first_cs_presentation_frame"] = 1685
video_file_name_to_sub_dicts["10-03-22_Test_29_3-2v3-3.mp4"]["first_cs_presentation_frame"] = 1669
video_file_name_to_sub_dicts["10-03-22_Test_22_6-1v6-3.mp4"]["first_cs_presentation_frame"] = 2055
video_file_name_to_sub_dicts["10-03-22_Test_32_4-1v4-4.mp4"]["first_cs_presentation_frame"] = 1669
video_file_name_to_sub_dicts["10-03-22_Test_34_5-1v5-4.mp4"]["first_cs_presentation_frame"] = 2008

In [ ]:
for video_relative_path in video_file_paths:
    video_file_name = os.path.basename(video_relative_path)
    if video_file_name in video_file_name_to_sub_dicts:
        first_cs_presentation_in_frames = video_file_name_to_sub_dicts[video_file_name]["first_cs_presentation_frame"]       
        estimated_vs_actual_first_cs_presentation_in_frames_difference = first_cs_presentation_in_frames - video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["estimated_cs_time_in_frames"].iloc[0]
        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["observed_cs_presentation_in_frame"] = (video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["estimated_cs_time_in_frames"] + estimated_vs_actual_first_cs_presentation_in_frames_difference).round(0).astype(int)
        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["observed_cs_presentation_in_seconds"] = (video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["observed_cs_presentation_in_frame"] / video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["fps"]).round(2)
        
        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["10_seconds_before_cs_presentation_in_frame"] = video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["observed_cs_presentation_in_frame"] - (10 * video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["fps"])
        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["10_seconds_before_cs_presentation_in_frame"] = (video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["10_seconds_before_cs_presentation_in_frame"]).round(0).astype(int)
        
        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["10_seconds_after_cs_presentation_in_frame"] = video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["observed_cs_presentation_in_frame"] + (10 * video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["fps"])
        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["10_seconds_after_cs_presentation_in_frame"] = (video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["10_seconds_after_cs_presentation_in_frame"]).round(0).astype(int)
        
        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["10_seconds_after_cs_presentation_in_seconds"] = video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["10_seconds_after_cs_presentation_in_frame"] / video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["fps"]
        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["10_seconds_before_cs_presentation_in_seconds"] = video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["10_seconds_before_cs_presentation_in_frame"] / video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["fps"]


In [ ]:
video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]

In [ ]:
video_file_name_to_sub_dicts["10-03-22_Test_13_2-2v2-4.mp4"]["cs_presentation_df"]

# Preparing for the inference

In [ ]:
for video_relative_path in video_file_paths:
    video_file_name = os.path.basename(video_relative_path)
    if video_file_name in video_file_name_to_sub_dicts:
        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["prediction_clip_output_dirname"] = video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"].apply(lambda x: os.path.join(os.path.dirname(x["video_full_path"]), Path(x["video_file_name"]).stem), axis=1)
        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["prediction_clip_output_full_path"] = video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"].apply(lambda x: os.path.join(x["prediction_clip_output_dirname"], "{}.tone_{}.predictions.slp".format(Path(x["video_file_name"]), x["tone_number"])), axis=1)


In [ ]:
all_cs_presentation_dataframe["video_full_path"].iloc[0]

In [ ]:
video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]

In [ ]:
video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["prediction_clip_output_full_path"].iloc[0]

In [ ]:
for video_relative_path in video_file_paths:
    video_file_name = os.path.basename(video_relative_path)
    if video_file_name in video_file_name_to_sub_dicts:
        first_cs_presentation_in_frames = video_file_name_to_sub_dicts[video_file_name]["first_cs_presentation_frame"]       
        estimated_vs_actual_first_cs_presentation_in_frames_difference = first_cs_presentation_in_frames - video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"]["estimated_cs_time_in_frames"].iloc[0]
        video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"].to_csv("./proc/{}.cs_presentation_frame_numbers.csv".format(video_file_name))

- Combining all the videos

In [ ]:
raise ValueError()

In [ ]:
all_cs_presentation_df_list = []
for video_relative_path in video_file_paths:
    video_file_name = os.path.basename(video_relative_path)
    if video_file_name in video_file_name_to_sub_dicts:
        all_cs_presentation_df_list.append(video_file_name_to_sub_dicts[video_file_name]["cs_presentation_df"])

In [ ]:
all_cs_presentation_df_list

In [ ]:
all_cs_presentation_dataframe = pd.concat(all_cs_presentation_df_list)

In [ ]:
all_cs_presentation_dataframe

In [ ]:
all_cs_presentation_dataframe["video_file_name"].unique()

In [ ]:
all_cs_presentation_dataframe.to_csv("./proc/cs_presentation_frame_numbers.csv")

- Trimming the video

In [ ]:
for index, row in all_cs_presentation_dataframe.iterrows():
    os.makedirs(row["tone_clip_output_dirname"], exist_ok=True)
    video_object = VideoFileClip(row["video_full_path"])
    cs_trimmed_video = video_object.subclip(row["10_seconds_before_cs_presentation_in_seconds"], row["10_seconds_after_cs_presentation_in_seconds"])
    cs_trimmed_video.to_videofile(row["tone_clip_output_full_path"])

In [ ]:
raise ValueError()

In [ ]:
video_object = VideoFileClip(video_path)

In [ ]:
video_object.duration

fixed func issue with: https://github.com/Zulko/moviepy/issues/1765


In [ ]:
video_path

In [ ]:
file_name = os.path.basename(video_path)
file_name_without_extention = os.path.splitext(file_name)[0]
file_name_without_extention = "_".join(file_name_without_extention.split())

In [ ]:
file_name_without_extention

In [ ]:
clip_output_dir = "./data/videos/tone_clips/"

In [ ]:
current_clip_output_dir = os.path.join(clip_output_dir, file_name_without_extention)
os.makedirs(current_clip_output_dir, exist_ok=True)

In [ ]:
for index, row in all_cs_presentation_dataframe.iterrows():
    cs_trimmed_video = video_object.subclip(row["estimated_cs_time_in_seconds"] - 5, row["estimated_cs_time_in_seconds"] + 15)
    cs_trimmed_file_name = "cs_number_{:02d}_date_{}.mp4".format(index + 1, file_name_without_extention)
    print(cs_trimmed_file_name)
    cs_trimmed_video.to_videofile(os.path.join(current_clip_output_dir, cs_trimmed_file_name))

In [ ]:
cs_trimmed_times_name = "cs_times_{}_.csv".format(file_name_without_extention)


In [ ]:
cs_presentation_time.to_csv(os.path.join(current_clip_output_dir, cs_trimmed_times_name))